In [1]:
import sagemaker
import boto3
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker.session import s3_input, Session

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
bucket_name = 'awsmlapp1'
my_region = boto3.session.Session().region_name
print(my_region)

us-east-2


In [3]:
s3 = boto3.resource('s3')
try:
    if my_region == 'us-east-2':
        s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': my_region})
        print('S3 Bucket Created')
except Exception as e:
    print('S3 error: ', e)

S3 error:  An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [4]:
prefix = 'xgboost'
output_path = 's3://{}/{}/output'.format(bucket_name, prefix)
print(output_path)

s3://awsmlapp1/xgboost/output


In [5]:
pth = "https://d1.awsstatic.com/tmt/build-train-deploy-machine-learning-model-sagemaker/bank_clean.27f01fbbdf43271788427f3682996ae29ceca05d.csv"

In [6]:
import pandas as pd
import urllib

In [7]:
try:
    urllib.request.urlretrieve(pth, "clean-data.csv")
    print('Success')
except Exception as e:
    print('Error: ', e)

Success


In [8]:
try:
    df = pd.read_csv('./clean-data.csv', index_col=0)
    print('Success')
except Exception as e:
    print('Error Loading data: ', e)

Success


In [9]:
import numpy as np

In [10]:
train, test = np.split(df.sample(frac=1, random_state=1), [int(0.7*len(df))])
print(train.shape, test.shape)

(28831, 61) (12357, 61)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [13]:
import os
pd.concat([train['y_yes'], train.drop(['y_no', 'y_yes'], axis = 1)], axis = 1).to_csv('train.csv', index = False, header = False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
s3_input_train = sagemaker.TrainingInput(s3_data = 's3://{}/{}/train'.format(bucket_name, prefix), content_type = 'csv')

In [15]:
pd.concat([test['y_yes'], test.drop(['y_no', 'y_yes'], axis = 1)], axis = 1).to_csv('test.csv', index = False, header = False)
boto3.Session().resource('s3').Bucket(bucket_name).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')
s3_input_test = sagemaker.TrainingInput(s3_data = 's3://{}/{}/test'.format(bucket_name, prefix), content_type = "csv")

In [16]:
container = get_image_uri(boto3.Session().region_name,
                          'xgboost',
                          repo_version = '1.0-1'
                         )

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [29]:
hyperparameters = {
    "max_depth" : "5",
    "eta":"0.2",
    "gamma":"4",
    "min_child_weight":"6",
    "subsample":"0.7",
    "objective":"binary:logistic",
    "num_round": "20",
}

In [30]:
estimator = sagemaker.estimator.Estimator(image_uri=container,
                                          hyperparameters = hyperparameters,
                                          role = sagemaker.get_execution_role(),
                                          train_instance_count = 1,
                                          train_instance_type = 'ml.m5.2xlarge',
                                          train_volume_sizes = 5,
                                          output_path = output_path,
                                          train_use_spot_instances = True,
                                          train_max_run = 300,
                                          train_max_wait = 600)
                                          

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [31]:
estimator.fit({'train': s3_input_train, 'validation': s3_input_test})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2024-01-31-19-59-43-627


2024-01-31 19:59:43 Starting - Starting the training job...
2024-01-31 19:59:58 Starting - Preparing the instances for training......
2024-01-31 20:00:55 Downloading - Downloading input data...
2024-01-31 20:01:36 Training - Training image download completed. Training in progress....
2024-01-31 20:02:12 Uploading - Uploading generated training model
2024-01-31 20:02:12 Completed - Training job completed
[2024-01-31 20:01:56.650 ip-10-0-179-230.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
I

In [32]:
xgb_predictor = estimator.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge')

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2024-01-31-20-06-40-500
INFO:sagemaker:Creating endpoint-config with name sagemaker-xgboost-2024-01-31-20-06-40-500
INFO:sagemaker:Creating endpoint with name sagemaker-xgboost-2024-01-31-20-06-40-500


-----!

In [35]:
from sagemaker.serializers import CSVSerializer

In [36]:
csv_serializer = CSVSerializer()
test_data_array = test.drop(['y_no', 'y_yes'], axis = 1).values
xgb_predictor.content_type = 'text/csv' # set the data type for an inference
xgb_predictor.serializer = csv_serializer # set the serializer type
predictions = xgb_predictor.predict(test_data_array).decode('utf-8') # predict!
predictions_array = np.fromstring(predictions[1:], sep=',') # and turn the prediction into an array
print(predictions_array.shape)

(12357,)


In [38]:
cm = pd.crosstab(index=test['y_yes'], columns=np.round(predictions_array), rownames=['Observed'], colnames=['Predicted'])
tn = cm.iloc[0,0]; fn = cm.iloc[1,0]; tp = cm.iloc[1,1]; fp = cm.iloc[0,1]; p = (tp+tn)/(tp+tn+fp+fn)*100
print("\n{0:<20}{1:<4.1f}%\n".format("Overall Classification Rate: ", p))
print("{0:<15}{1:<15}{2:>8}".format("Predicted", "No Purchase", "Purchase"))
print("Observed")
print("{0:<15}{1:<2.0f}% ({2:<}){3:>6.0f}% ({4:<})".format("No Purchase", tn/(tn+fn)*100,tn, fp/(tp+fp)*100, fp))
print("{0:<16}{1:<1.0f}% ({2:<}){3:>7.0f}% ({4:<}) \n".format("Purchase", fn/(tn+fn)*100,fn, tp/(tp+fp)*100, tp))


Overall Classification Rate: 90.0%

Predicted      No Purchase    Purchase
Observed
No Purchase    91% (10866)    36% (143)
Purchase        9% (1090)     64% (258) 



In [39]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)
bucket_to_delete = boto3.resource('s3').Bucket(bucket_name)
bucket_to_delete.objects.all().delete()

See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
INFO:sagemaker:Deleting endpoint with name: sagemaker-xgboost-2024-01-31-20-06-40-500


[{'ResponseMetadata': {'RequestId': 'JHY729J612V9TM90',
   'HostId': 'EODajOsbDRHXVUhXxk6OkGrruV+qDJg4hLXjwwL5Fm9zebmYHJmAuPvM7XMGO0IuIUCt04FOiGQ=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'EODajOsbDRHXVUhXxk6OkGrruV+qDJg4hLXjwwL5Fm9zebmYHJmAuPvM7XMGO0IuIUCt04FOiGQ=',
    'x-amz-request-id': 'JHY729J612V9TM90',
    'date': 'Wed, 31 Jan 2024 20:18:39 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'xgboost/output/sagemaker-xgboost-2024-01-31-19-53-22-490/profiler-output/system/incremental/2024013119/1706730900.algo-1.json'},
   {'Key': 'xgboost/output/sagemaker-xgboost-2024-01-31-19-59-43-627/debug-output/index/000000000/000000000010_worker_0.json'},
   {'Key': 'xgboost/output/sagemaker-xgboost-2024-01-31-19-59-43-627/debug-output/claim.smd'},
   {'Key': 'xgboost/output/sagemaker-xgboost-2024-01-31-19-59-43-627/output/model.tar.gz'},
   